In [1]:
import pandas as pd
import numpy as np
import torch
from torch import nn



In [2]:
import torch.nn as nn
import torch.nn.functional as F

In [3]:
data = pd.read_csv('./test_tables/covid19MH_20201027_geral_MLdata.csv', sep = ';')

/home/guilherme/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,16,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,205,206,207,208,243,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,270,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,325) have mixed types. Specify dtype opt

In [4]:
list(data.columns)

['ID_covid',
 'sample',
 'date_created',
 'consent',
 'sgID',
 'date_birth',
 'region',
 'state_br',
 'city_br',
 'age_y_given',
 'sex',
 'education',
 'marital_status',
 'ethnicity',
 'work_type',
 'Occupation_other',
 'health_category',
 'working_hours',
 'religion',
 'religion_other',
 'cceb_automobile',
 'cceb_domestic_servant',
 'cceb_washing_machine',
 'cceb_bathroom',
 'cceb_refrigerator',
 'cceb_freezer',
 'cceb_personal_computers',
 'cceb_dishwasher',
 'cceb_DVDplayer',
 'cceb_microwave_oven',
 'cceb_motorcycle',
 'cceb_clothes_dryer',
 'cceb_piped_water',
 'cceb_paved_street',
 'cceb_drinking_water',
 'cceb_electricity',
 'cceb_total',
 'cceb_class',
 'residents_65',
 'residents_1464',
 'residents_614',
 'residents_05',
 'household_income',
 'education_householder',
 'education_mother',
 'education_father',
 'residents_total',
 'fever',
 'chills',
 'headache',
 'myalgia',
 'cough',
 'breathing_difficulty',
 'dizziness',
 'coryza',
 'sore_throat',
 'chest_pressure',
 'bluish_l

In [5]:
new_feat = []#list(data.columns)[42:141]
new_feat += ['WHOQoL_2','WHOQoL_3', 'WHOQoL_4', 'WHOQoL_6', 'WHOQoL_10', 
              'WHOQoL_15', 'WHOQoL_16', 'WHOQoL_21','WHOQoL_23','WHOQoL_25','WHOQoL_26',
             'BSI_9', 'BSI_12', 'BSI_23','BSI_37','BSI_38', 'BSI_45','BSI_49'
            ]

In [6]:
result_cols = []
for col in data.columns:
    if 'whoqol' in col:
        result_cols.append(col)
    elif 'BSI_' in col and len(col)>6:
        result_cols.append(col)

In [7]:
bsi = []
whoqol = []
for col in data.columns:
    if 'whoqol' in col:
        whoqol.append(col)
    elif 'BSI_' in col and len(col)>6:
        bsi.append(col)
bsi_feat = []
whoqol_feat = []
for col in data.columns:
    if 'WHOQoL' in col:
        whoqol_feat.append(col)
    elif 'BSI_' in col and len(col)<=6:
        bsi_feat.append(col)

In [8]:
bsi_feat = []
whoqol_feat = []
for col in data.columns:
    if 'WHOQoL' in col:
        whoqol_feat.append(col)
    elif 'BSI_' in col and len(col)<=6:
        bsi_feat.append(col)

In [9]:

data[result_cols] = data[result_cols].replace(' ', np.nan)
data = data.replace(' ', np.nan)


In [10]:
data.dropna(subset = result_cols, inplace = True)

In [11]:
data.drop(['ID_covid', 'sample', 'date_created','consent', 'sgID', 'date_birth', 'region','city_br','Occupation_other', 'working_hours','religion_other', 'cceb_class',
'social_support_n_family','social_support_n_friends'],axis = 1, inplace = True)

In [12]:
data[bsi] = data[bsi].apply(lambda x: x.str.replace(',','.'))
data[whoqol] = data[whoqol].apply(lambda x: x.str.replace(',','.'))

In [13]:
data[bsi_feat] = data[bsi_feat].astype(float)

data[whoqol_feat] = data[whoqol_feat].astype(float)

In [14]:
for col in data.columns:
    try:
        data[col] = data[col].astype(float)
    except:
        (print(col))

age_y_given
iesrintrusion
iesravoidance
iesrhyperarousal
iesrtotal
height_cm


In [15]:
import numpy as np # linear algebra
from numpy.random import seed
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from keras.layers import Input, merge
from keras.layers.core import  Dense, Dropout
from keras.models import Model
from tensorflow.python.client import device_lib
import keras
from keras import optimizers
from keras.layers.advanced_activations import LeakyReLU
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [16]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split

In [17]:
for col in data.columns:
    if data[col].nunique()==1:
        data.drop(col,inplace = True,axis=1)

In [18]:
data.fillna(-1, inplace = True)
Y1 = data['BSI_Somatization_EB']
Y2 = data[whoqol]
#data['mean'] = np.round(data['mean']/10)

#Y2 = data['"ARTERIAL_DIAS"']
data.drop(whoqol + bsi ,axis = 1, inplace = True)
X = data[bsi_feat]
X2 = data[new_feat]

In [19]:
#Y2 = Y2['whoqol_Physical']
#Y2 = Y2['whoqol_Psychological']
#Y2 = Y2['whoqol_SocialRelations']
Y2 = Y2['whoqol_Environment']


In [20]:
Y1 = Y1/np.max(Y1)
#Y1 = Y1['BSI_GSI']
Y2 = Y2/np.max(Y2)
#Y2 = Y2.mean(axis=1)

# Train 1

In [21]:
X_train, X_test, y_train, y_test = train_test_split(
     X, Y1, test_size=0.33, random_state=42)

In [22]:
X2_train, X2_test, y2_train, y2_test = train_test_split(
     X2, Y2, test_size=0.33, random_state=42)

In [23]:
encoding_dim = 1
ncol = len(X2_train.columns)
keras.backend.clear_session()

In [24]:
keras.backend.clear_session()
input_dim = Input(shape = (ncol, ))
# Encoder Layers


encoded1 = Dense(200, activation = 'tanh')(input_dim)
drop1 = Dropout(0.1)(encoded1)
encoded2 = Dense(100, activation = 'tanh')(encoded1)
encoded3 = Dense(100, activation = 'tanh')(encoded2)
encoded4 = Dense(100, activation = 'tanh')(encoded3)
encoded5 = Dense(100, activation = 'tanh')(encoded4)
decoded13 = Dense(1, activation = 'sigmoid')(encoded5)

# Combine Encoder and Deocder layers
autoencoder = Model(inputs = input_dim, outputs = [decoded13])

# Compile the Model
autoencoder.compile(optimizer = 'adam', loss = 'mean_absolute_error')

In [25]:
es = keras.callbacks.EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=1000,
                              verbose=0, mode='auto',
                                  restore_best_weights=True)




callbacks = [es]
autoencoder.fit(x = X2_train, y = y_train, epochs = 1500, 
                batch_size = 128, shuffle = True,
                 validation_data=(X2_test, y_test),
               callbacks = callbacks
                
               )

Epoch 1/1500
29/29 [==============================] - 0s 6ms/step - loss: 0.0963 - val_loss: 0.0621
Epoch 2/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0620 - val_loss: 0.0683
Epoch 3/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0613 - val_loss: 0.0579
Epoch 4/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0583 - val_loss: 0.0580
Epoch 5/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0597 - val_loss: 0.0586
Epoch 6/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0573 - val_loss: 0.0583
Epoch 7/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0571 - val_loss: 0.0656
Epoch 8/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0569 - val_loss: 0.0589
Epoch 9/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0566 - val_loss: 0.0567
Epoch 10/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0568 - val_loss: 0.0622

29/29 [==============================] - 0s 2ms/step - loss: 0.0410 - val_loss: 0.0637
Epoch 83/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0416 - val_loss: 0.0612
Epoch 84/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0412 - val_loss: 0.0614
Epoch 85/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0400 - val_loss: 0.0681
Epoch 86/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0398 - val_loss: 0.0633
Epoch 87/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0409 - val_loss: 0.0629
Epoch 88/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0394 - val_loss: 0.0628
Epoch 89/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0399 - val_loss: 0.0634
Epoch 90/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0394 - val_loss: 0.0616
Epoch 91/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0380 - val_loss: 0.0619
Epoc

29/29 [==============================] - 0s 2ms/step - loss: 0.0197 - val_loss: 0.0669
Epoch 163/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0199 - val_loss: 0.0660
Epoch 164/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0193 - val_loss: 0.0676
Epoch 165/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0193 - val_loss: 0.0669
Epoch 166/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0206 - val_loss: 0.0675
Epoch 167/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0203 - val_loss: 0.0683
Epoch 168/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0184 - val_loss: 0.0669
Epoch 169/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0187 - val_loss: 0.0669
Epoch 170/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0181 - val_loss: 0.0665
Epoch 171/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0182 - val_loss: 0.

29/29 [==============================] - 0s 3ms/step - loss: 0.0137 - val_loss: 0.0683
Epoch 243/1500
29/29 [==============================] - 0s 3ms/step - loss: 0.0137 - val_loss: 0.0672
Epoch 244/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0123 - val_loss: 0.0670
Epoch 245/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0144 - val_loss: 0.0670
Epoch 246/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0141 - val_loss: 0.0675
Epoch 247/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0140 - val_loss: 0.0671
Epoch 248/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0133 - val_loss: 0.0671
Epoch 249/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0126 - val_loss: 0.0667
Epoch 250/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0125 - val_loss: 0.0666
Epoch 251/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0132 - val_loss: 0.

29/29 [==============================] - 0s 2ms/step - loss: 0.0115 - val_loss: 0.0689
Epoch 323/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0110 - val_loss: 0.0670
Epoch 324/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0106 - val_loss: 0.0676
Epoch 325/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0107 - val_loss: 0.0669
Epoch 326/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0102 - val_loss: 0.0674
Epoch 327/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0099 - val_loss: 0.0669
Epoch 328/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0100 - val_loss: 0.0658
Epoch 329/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0105 - val_loss: 0.0664
Epoch 330/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0116 - val_loss: 0.0669
Epoch 331/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0107 - val_loss: 0.

...

29/29 [==============================] - 0s 2ms/step - loss: 0.0066 - val_loss: 0.0658
Epoch 643/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0067 - val_loss: 0.0659
Epoch 644/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0067 - val_loss: 0.0649
Epoch 645/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0070 - val_loss: 0.0659
Epoch 646/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0067 - val_loss: 0.0660
Epoch 647/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0067 - val_loss: 0.0658
Epoch 648/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0065 - val_loss: 0.0655
Epoch 649/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0067 - val_loss: 0.0657
Epoch 650/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0065 - val_loss: 0.0663
Epoch 651/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0067 - val_loss: 0.

29/29 [==============================] - 0s 2ms/step - loss: 0.0065 - val_loss: 0.0661
Epoch 723/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0066 - val_loss: 0.0650
Epoch 724/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0066 - val_loss: 0.0654
Epoch 725/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0063 - val_loss: 0.0655
Epoch 726/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0068 - val_loss: 0.0656
Epoch 727/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0066 - val_loss: 0.0656
Epoch 728/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0066 - val_loss: 0.0663
Epoch 729/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0070 - val_loss: 0.0653
Epoch 730/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0068 - val_loss: 0.0652
Epoch 731/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0061 - val_loss: 0.

29/29 [==============================] - 0s 2ms/step - loss: 0.0060 - val_loss: 0.0653
Epoch 803/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0062 - val_loss: 0.0660
Epoch 804/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0060 - val_loss: 0.0648
Epoch 805/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0063 - val_loss: 0.0647
Epoch 806/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0062 - val_loss: 0.0652
Epoch 807/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0059 - val_loss: 0.0656
Epoch 808/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0059 - val_loss: 0.0654
Epoch 809/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0056 - val_loss: 0.0656
Epoch 810/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0059 - val_loss: 0.0649
Epoch 811/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0062 - val_loss: 0.

29/29 [==============================] - 0s 2ms/step - loss: 0.0054 - val_loss: 0.0649
Epoch 883/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0054 - val_loss: 0.0655
Epoch 884/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0054 - val_loss: 0.0655
Epoch 885/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0053 - val_loss: 0.0653
Epoch 886/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0055 - val_loss: 0.0655
Epoch 887/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0057 - val_loss: 0.0653
Epoch 888/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0052 - val_loss: 0.0654
Epoch 889/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0054 - val_loss: 0.0653
Epoch 890/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0053 - val_loss: 0.0654
Epoch 891/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0054 - val_loss: 0.

29/29 [==============================] - 0s 2ms/step - loss: 0.0055 - val_loss: 0.0650
Epoch 963/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0051 - val_loss: 0.0647
Epoch 964/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0051 - val_loss: 0.0650
Epoch 965/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0050 - val_loss: 0.0652
Epoch 966/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0050 - val_loss: 0.0652
Epoch 967/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0054 - val_loss: 0.0649
Epoch 968/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0053 - val_loss: 0.0654
Epoch 969/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0054 - val_loss: 0.0651
Epoch 970/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0052 - val_loss: 0.0650
Epoch 971/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0054 - val_loss: 0.

In [26]:
pred = autoencoder.predict(X2_test)

In [27]:
pred[:5]

array([[0.07451487],
       [0.07762942],
       [0.04264787],
       [0.18289927],
       [0.44486928]], dtype=float32)

In [28]:
np.mean(y_test)

0.17713419435220926

In [29]:
print(r2_score(pred,y_test))
mean_absolute_error(pred,y_test)

0.7757035674021516


0.05670512407476525

##### Train 2

In [30]:
encoding_dim = 1
ncol = len(X2.columns)
keras.backend.clear_session()

In [31]:
keras.backend.clear_session()
input_dim = Input(shape = (ncol, ))
# Encoder Layers

encoded1 = Dense(200, activation = 'tanh')(input_dim)
drop1 = Dropout(0.1)(encoded1)
encoded2 = Dense(100, activation = 'tanh')(encoded1)
encoded3 = Dense(100, activation = 'tanh')(encoded2)
encoded4 = Dense(100, activation = 'tanh')(encoded3)
encoded5 = Dense(100, activation = 'tanh')(encoded4)
decoded13 = Dense(1, activation = 'sigmoid')(encoded5)

# Combine Encoder and Deocder layers
autoencoder2 = Model(inputs = input_dim, outputs = [decoded13])

# Compile the Model
autoencoder2.compile(optimizer = 'adam', loss = 'mean_absolute_error')

In [32]:
es = keras.callbacks.EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=1000,
                              verbose=0, mode='auto',
                                  restore_best_weights=True)




callbacks = [es]
autoencoder2.fit(x = X2_train, y = y2_train, epochs = 1500, 
                batch_size = 128, shuffle = True,
                 validation_data=(X2_test, y2_test),
               callbacks = callbacks
                
               )

Epoch 1/1500
29/29 [==============================] - 0s 5ms/step - loss: 0.1043 - val_loss: 0.0853
Epoch 2/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0831 - val_loss: 0.0768
Epoch 3/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0746 - val_loss: 0.0722
Epoch 4/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0737 - val_loss: 0.0755
Epoch 5/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0717 - val_loss: 0.0852
Epoch 6/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0722 - val_loss: 0.0726
Epoch 7/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0709 - val_loss: 0.0702
Epoch 8/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0723 - val_loss: 0.0767
Epoch 9/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0693 - val_loss: 0.0707
Epoch 10/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0697 - val_loss: 0.0821

29/29 [==============================] - 0s 2ms/step - loss: 0.0473 - val_loss: 0.0730
Epoch 83/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0469 - val_loss: 0.0735
Epoch 84/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0463 - val_loss: 0.0744
Epoch 85/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0491 - val_loss: 0.0737
Epoch 86/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0463 - val_loss: 0.0743
Epoch 87/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0446 - val_loss: 0.0751
Epoch 88/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0442 - val_loss: 0.0738
Epoch 89/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0438 - val_loss: 0.0749
Epoch 90/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0440 - val_loss: 0.0754
Epoch 91/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0439 - val_loss: 0.0750
Epoc

29/29 [==============================] - 0s 3ms/step - loss: 0.0224 - val_loss: 0.0799
Epoch 163/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0242 - val_loss: 0.0802
Epoch 164/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0229 - val_loss: 0.0801
Epoch 165/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0227 - val_loss: 0.0803
Epoch 166/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0222 - val_loss: 0.0797
Epoch 167/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0230 - val_loss: 0.0799
Epoch 168/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0210 - val_loss: 0.0794
Epoch 169/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0227 - val_loss: 0.0792
Epoch 170/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0213 - val_loss: 0.0809
Epoch 171/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0223 - val_loss: 0.

29/29 [==============================] - 0s 2ms/step - loss: 0.0163 - val_loss: 0.0802
Epoch 243/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0152 - val_loss: 0.0801
Epoch 244/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0152 - val_loss: 0.0807
Epoch 245/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0149 - val_loss: 0.0802
Epoch 246/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0154 - val_loss: 0.0803
Epoch 247/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0151 - val_loss: 0.0803
Epoch 248/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0145 - val_loss: 0.0803
Epoch 249/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0156 - val_loss: 0.0809
Epoch 250/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0164 - val_loss: 0.0809
Epoch 251/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0155 - val_loss: 0.

29/29 [==============================] - 0s 2ms/step - loss: 0.0119 - val_loss: 0.0802
Epoch 323/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0139 - val_loss: 0.0807
Epoch 324/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0131 - val_loss: 0.0803
Epoch 325/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0127 - val_loss: 0.0809
Epoch 326/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0116 - val_loss: 0.0802
Epoch 327/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0117 - val_loss: 0.0810
Epoch 328/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0124 - val_loss: 0.0804
Epoch 329/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0127 - val_loss: 0.0805
Epoch 330/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0118 - val_loss: 0.0807
Epoch 331/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0116 - val_loss: 0.

...

29/29 [==============================] - 0s 2ms/step - loss: 0.0085 - val_loss: 0.0791
Epoch 643/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0088 - val_loss: 0.0795
Epoch 644/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0092 - val_loss: 0.0792
Epoch 645/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0087 - val_loss: 0.0788
Epoch 646/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0087 - val_loss: 0.0785
Epoch 647/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0082 - val_loss: 0.0795
Epoch 648/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0083 - val_loss: 0.0790
Epoch 649/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0084 - val_loss: 0.0795
Epoch 650/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0083 - val_loss: 0.0788
Epoch 651/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0081 - val_loss: 0.

29/29 [==============================] - 0s 2ms/step - loss: 0.0083 - val_loss: 0.0791
Epoch 723/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0084 - val_loss: 0.0791
Epoch 724/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0077 - val_loss: 0.0793
Epoch 725/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0081 - val_loss: 0.0789
Epoch 726/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0084 - val_loss: 0.0791
Epoch 727/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0084 - val_loss: 0.0789
Epoch 728/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0077 - val_loss: 0.0792
Epoch 729/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0080 - val_loss: 0.0794
Epoch 730/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0077 - val_loss: 0.0788
Epoch 731/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0078 - val_loss: 0.

29/29 [==============================] - 0s 2ms/step - loss: 0.0074 - val_loss: 0.0788
Epoch 803/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0070 - val_loss: 0.0787
Epoch 804/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0074 - val_loss: 0.0786
Epoch 805/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0070 - val_loss: 0.0787
Epoch 806/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0080 - val_loss: 0.0788
Epoch 807/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0082 - val_loss: 0.0783
Epoch 808/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0083 - val_loss: 0.0789
Epoch 809/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0079 - val_loss: 0.0789
Epoch 810/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0075 - val_loss: 0.0783
Epoch 811/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0079 - val_loss: 0.

29/29 [==============================] - 0s 2ms/step - loss: 0.0068 - val_loss: 0.0788
Epoch 883/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0075 - val_loss: 0.0789
Epoch 884/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0071 - val_loss: 0.0789
Epoch 885/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0074 - val_loss: 0.0784
Epoch 886/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0070 - val_loss: 0.0788
Epoch 887/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0074 - val_loss: 0.0789
Epoch 888/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0072 - val_loss: 0.0789
Epoch 889/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0067 - val_loss: 0.0786
Epoch 890/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0070 - val_loss: 0.0791
Epoch 891/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0069 - val_loss: 0.

29/29 [==============================] - 0s 2ms/step - loss: 0.0069 - val_loss: 0.0787
Epoch 963/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0067 - val_loss: 0.0795
Epoch 964/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0069 - val_loss: 0.0786
Epoch 965/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0069 - val_loss: 0.0793
Epoch 966/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0070 - val_loss: 0.0788
Epoch 967/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0068 - val_loss: 0.0785
Epoch 968/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0075 - val_loss: 0.0787
Epoch 969/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0075 - val_loss: 0.0790
Epoch 970/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0074 - val_loss: 0.0790
Epoch 971/1500
29/29 [==============================] - 0s 2ms/step - loss: 0.0078 - val_loss: 0.

In [33]:
pred2 = autoencoder2.predict(X2_test)

In [34]:
pred2[:5]

array([[0.79278195],
       [0.789006  ],
       [0.66530806],
       [0.15181437],
       [0.28259662]], dtype=float32)

In [35]:
y2_test.head()

1859    0.81250
691     0.81250
1858    0.59375
1101    0.15625
5009    0.25000
Name: whoqol_Environment, dtype: float64

In [36]:

print(r2_score(pred2,y2_test))
mean_absolute_error(pred2,y2_test)

0.7068062139029667


0.06793729844272071

# Counterfactual

In [37]:
encoded_train2 = pd.DataFrame(autoencoder2.predict(X2_test))
encoded_train2 = encoded_train2.add_prefix('feature_')
encoded_train = pd.DataFrame(autoencoder.predict(X2_test))
encoded_train = encoded_train.add_prefix('feature_')

In [38]:
error_test = y_test.values - encoded_train['feature_0']
error_test2 = y2_test.values - encoded_train2['feature_0']

In [39]:
encoded_train2 = pd.DataFrame(autoencoder2.predict(X2_train))
encoded_train2 = encoded_train2.add_prefix('feature_')
encoded_train = pd.DataFrame(autoencoder.predict(X2_train))
encoded_train = encoded_train.add_prefix('feature_')

In [40]:
error = y_train.values - encoded_train['feature_0']
error2 = y2_train.values - encoded_train2['feature_0']

In [41]:
X2_train.head()

,WHOQoL_2,WHOQoL_3,WHOQoL_4,WHOQoL_6,WHOQoL_10,WHOQoL_15,WHOQoL_16,WHOQoL_21,WHOQoL_23,WHOQoL_25,WHOQoL_26,BSI_9,BSI_12,BSI_23,BSI_37,BSI_38,BSI_45,BSI_49
2513,4.0,1.0,1.0,5.0,4.0,4.0,4.0,4.0,5.0,4.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2627,1.0,3.0,3.0,1.0,2.0,4.0,1.0,1.0,5.0,5.0,4.0,2.0,3.0,4.0,4.0,4.0,4.0,0.0
5005,3.0,1.0,1.0,4.0,3.0,4.0,3.0,2.0,3.0,1.0,3.0,0.0,2.0,0.0,0.0,2.0,1.0,0.0
3822,3.0,2.0,5.0,4.0,4.0,3.0,3.0,1.0,3.0,4.0,5.0,0.0,3.0,2.0,1.0,3.0,2.0,1.0
4510,4.0,3.0,3.0,5.0,3.0,3.0,2.0,3.0,2.0,4.0,4.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0


In [42]:
X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
X2_train['y'] = y_train.values
X2_test['y'] = y_test.values
X2_train['error'] = error.values
X2_test['error'] = error_test.values


X2_train['error2'] = error2.values
X2_test['error2'] = error_test2.values
X2_train['y2'] = y2_train.values
X2_test['y2'] = y2_test.values

/home/guilherme/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/guilherme/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/guilherme/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

In [43]:
#df_encoder_train = X_train.merge(X2_train,left_index=True,right_index=True)
#df_encoder_test = X_test.merge(X2_test,left_index=True,right_index=True)

In [44]:
df_encoder_train = X2_train.copy()#.merge(X2_train,left_index=True,right_index=True)
df_encoder_test = X2_test.copy()#.merge(X2_test,left_index=True,right_index=True)


In [45]:
df_encoder_train = X2_train.copy()#.merge(X2_train,left_index=True,right_index=True)
df_encoder_test = X2_test.copy()#.merge(X2_test,left_index=True,right_index=True)
df_outs_train = df_encoder_train[['y','y2']]
df_outs_test = df_encoder_test[['y','y2']]
df_features_train = df_encoder_train.drop(['error','error2','y','y2'], axis = 1)
df_features_test = df_encoder_test.drop(['error','error2','y','y2'], axis = 1)

In [46]:
df_encoder = df_encoder_train.append(df_encoder_test)

In [47]:
df_encoder['y1'] = df_encoder['y']

In [48]:
df_encoder['y']=df_encoder['y']<0.15

In [49]:
df_encoder.to_csv('./mace/mace/df_encoder_env.csv', index=False)

In [51]:
import numpy as np # linear algebra
from numpy.random import seed
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from keras.layers import Input, Dense, merge
from keras.models import Model
from tensorflow.python.client import device_lib
import keras
from keras import optimizers
from keras.layers.advanced_activations import LeakyReLU

In [52]:
encoding_dim = 1
ncol = len(df_encoder_train.columns)
keras.backend.clear_session()

In [53]:
class MyModel(nn.Module):
    def __init__(self, ncol, ncol2):
        super(MyModel, self).__init__()
        
        
        self.fc1 = nn.Linear(ncol, 15)
        self.fc2 = nn.Linear(15, 5)
        self.fc3 = nn.Linear(5, 1)
        self.fc4 = nn.Linear(1 + ncol2, 100)
        #self.droput = nn.Dropout(0.4)
        self.fc5 = nn.Linear(100, 100)
        self.fc6 = nn.Linear(100, 30)
        self.fc7 = nn.Linear(30, 1)
        self.fc8 = nn.Linear(30, 1)
        
    def forward(self, data1, data2):
        x1 = data1
        x2 = data2
        
        x = F.relu(self.fc1(x1))
        x = F.relu(self.fc2(x))
        x = (self.fc3(x))
        x = torch.cat((x, x2), dim=1)
        x = F.relu(self.fc4(x))
        #x = self.droput(x)
        x = F.relu(self.fc5(x))
        x = F.relu(self.fc6(x))
        y1 = self.fc7(x)
        y2 = self.fc8(x)
        return y1,y2
    
    

In [54]:
model = MyModel(ncol,ncol-4)
criterion = nn.L1Loss()
optimizer = torch.optim.ASGD(model.parameters())

In [55]:
df_encoder_train = df_encoder_train.sample(frac=1,random_state = 0)
df_features_train = df_features_train.sample(frac=1,random_state = 0)
df_outs_train = df_outs_train.sample(frac=1,random_state = 0)


In [62]:
df_encoder_train_dl = torch.utils.data.DataLoader(df_encoder_train.as_matrix(),batch_size=32, shuffle=False)
df_features_train_dl = torch.utils.data.DataLoader(df_features_train.as_matrix(),batch_size=32, shuffle=False)
Y1 = torch.utils.data.DataLoader(df_outs_train['y'].values ,batch_size=32, shuffle=False)
Y2 = torch.utils.data.DataLoader(df_outs_train['y2'].values ,batch_size=32, shuffle=False)

/home/guilherme/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/home/guilherme/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [63]:
for epoch in range(10):
    loss_n = []
    for  df_encoder_train_b,df_features_train_b, Y1_b, Y2_b  in zip(df_encoder_train_dl, df_features_train_dl, Y1, Y2):

        optimizer.zero_grad()
        # Forward pass
        y1,y2 = model(df_encoder_train_b.float(), df_features_train_b.float())
        # Compute Loss
        loss1 = criterion(y1, Y1_b.float())
        loss2 = criterion(y2,Y2_b.float() )
        
        
        loss = (loss1 + loss2)/2
        loss_n.append(loss.item())
        # Backward pass
        loss.backward()
        optimizer.step()
        
    loss_n = sum(loss_n)/len(loss_n)
    print('Epoch {}: train loss: {}'.format(epoch, loss_n))
    

Epoch 0: train loss: 0.807952839110432
Epoch 1: train loss: 0.14357509772325383
Epoch 2: train loss: 0.141489888123911
Epoch 3: train loss: 0.14075014425505852
Epoch 4: train loss: 0.14038093201816082
Epoch 5: train loss: 0.1401669737593881
Epoch 6: train loss: 0.14003377015991458
Epoch 7: train loss: 0.139945058344767
Epoch 8: train loss: 0.13989103816706558
Epoch 9: train loss: 0.13983689852315803


In [66]:
torch.save(model.state_dict(), './mace/mace/state')

In [65]:
model(df_encoder_train_b.float(), df_features_train_b.float())

(tensor([[0.0931],
         [0.0940]], grad_fn=<AddmmBackward>), tensor([[0.5976],
         [0.6151]], grad_fn=<AddmmBackward>))

In [ ]:
stop

In [ ]:
df_encoder_test_dl = torch.utils.data.DataLoader(df_encoder_test.as_matrix(),batch_size=len(df_encoder_test), shuffle=False)
df_features_test_dl = torch.utils.data.DataLoader(df_features_test.as_matrix(),batch_size=len(df_encoder_test), shuffle=False)
Y1_test = torch.utils.data.DataLoader(df_outs_test['y'].values ,batch_size=len(df_encoder_test), shuffle=False)
Y2_test = torch.utils.data.DataLoader(df_outs_test['y2'].values ,batch_size=len(df_encoder_test), shuffle=False)

In [ ]:
for  df_encoder_test_b,df_features_test_b, Y1_testb, Y2_testb  in zip(df_encoder_test_dl, df_features_test_dl, Y1_test, Y2_test):

        optimizer.zero_grad()
        # Forward pass
        y1,y2 = model(df_encoder_test_b.float(), df_features_test_b.float())
        loss1 = criterion(y1, Y1_testb.float())
        loss2 = criterion(y2,Y2_testb.float() )
        
        loss = (loss1 + loss2)/2
print(loss)      

In [ ]:
df_features_test_b.float()[0]

In [ ]:
y1,y2 = model(df_encoder_test_b.float()[0].view(1,-1), df_features_test_b.float()[0].view(1,-1))

In [ ]:
y1.squeeze().item()


In [ ]:
keras.backend.clear_session()
input_dim = Input(shape = (ncol, ))
input_dim2 = Input(shape = (ncol - 4, ))
# Encoder Layers
encoded4 = Dense(15, activation = 'relu')(input_dim2)
#dropen = Dropout(0.0)(encoded4)
encoded5 = Dense(10, activation = 'relu')(encoded4)
encoded6 = Dense(5, activation = 'relu')(encoded5)
encoded13 = Dense(encoding_dim, activation = 'linear')(encoded6)
merged = keras.layers.concatenate([encoded13, input_dim2], axis=-1)
# Decoder Layers
drop = Dropout(0.0)(merged)
decoded1 = Dense(30, activation = 'relu')(drop)
drop1 = Dropout(0.1)(decoded1)
decoded2 = Dense(30, activation = 'relu')(drop1)
decoded13 = Dense(2, activation = 'sigmoid')(decoded2)
counter = Model(inputs = [input_dim, input_dim2], outputs = [decoded13])
optimizer = keras.optimizers.Adam(lr= 0.0001)
# Compile the Model
counter.compile(optimizer = 'adam', loss = 'mean_absolute_error')

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=100,
                              verbose=0, mode='auto',
                                  restore_best_weights=True)


callbacks = [es]
counter.fit(x = [df_encoder_train, df_features_train], y = df_outs_train, epochs = 4500, 
                batch_size = 128, shuffle = True,
                validation_data=([df_encoder_test,df_features_test],df_outs_test),
               callbacks = callbacks
                
               )

In [ ]:
#counter = Model(inputs = [input_dim,input_dim2], outputs = encoded13)
counter = Model(inputs = [input_dim,input_dim2], outputs = decoded13)
encoded_input = Input(shape = (encoding_dim, ))

In [ ]:
encoded_test = pd.DataFrame(counter.predict([df_encoder_test,df_features_test]))
encoded_test = encoded_test.add_prefix('feature_')

In [ ]:
(encoded_test['feature_0']).sample(5)

In [ ]:
mean_absolute_error(encoded_test['feature_0'], df_outs_test['y']), mean_absolute_error(encoded_test['feature_1'], df_outs_test['y2'])

In [ ]:
stop

In [ ]:
r2_score(encoded_test['feature_0'], df_outs_test['y']), r2_score(encoded_test['feature_1'], df_outs_test['y2'])

In [ ]:
encoded_train = pd.DataFrame(counter.predict([df_encoder_train,df_features_train]))
encoded_train = encoded_train.add_prefix('feature_')

In [ ]:
mean_absolute_error(encoded_train['feature_0'], df_outs_train['y']), mean_absolute_error(encoded_train['feature_1'], df_outs_train['y2'])

In [ ]:
r2_score(encoded_train['feature_0'], df_outs_train['y']), r2_score(encoded_train['feature_1'], df_outs_train['y2'])

ind = 59
encoded_train.iloc[ind]


In [ ]:
new_feat_print = []#list(data.columns)[42:141]
new_feat_print +=  ['WHOQoL_2','WHOQoL_3', 'WHOQoL_4', 'WHOQoL_6', 'WHOQoL_10', 
              'WHOQoL_15', 'WHOQoL_16', 'WHOQoL_21','WHOQoL_23','WHOQoL_25','WHOQoL_26',
             #'BSI_9', 'BSI_12', 'BSI_23','BSI_37','BSI_38', 'BSI_45','BSI_49'
                   ]

In [ ]:
val = 5

In [ ]:
for feata in new_feat_print:
    #print(feata)
    value = val
    df_features_train = df_features_train[df_features_train[feata] != value]
    df_encoder_train = df_encoder_train[df_encoder_train[feata]!=value]
    
    counter = Model(inputs = [input_dim,input_dim2], outputs = decoded13)
    encoded_train = pd.DataFrame(counter.predict([df_encoder_train,df_features_train]))
    encoded_train = encoded_train.add_prefix('feature_')
    f0_a = encoded_train['feature_0']
    f1_a = encoded_train['feature_1']
    #print(encoded_train['feature_0'].median())
    #print(encoded_train['feature_1'].median())
    df_features_train[feata] = value
    counter = Model(inputs = [input_dim,input_dim2], outputs = decoded13)
    encoded_train = pd.DataFrame(counter.predict([df_encoder_train,df_features_train]))
    encoded_train = encoded_train.add_prefix('feature_')
    f0_d = encoded_train['feature_0']
    f1_d = encoded_train['feature_1']
    #f0r = f0_d - f0_a 
    f0r = f1_d - f1_a 
    pup = len(f0r[f0r>0.001])/len(f0r)
    pdo = len(f0r[f0r<-0.001])/len(f0r)
    peq = len(f0r[abs(f0r)<0.001])/len(f0r)
    
    print (pup)
    #print (pdo)
    #print (peq)
    df_encoder_train = X2_train.copy()#.merge(X2_train,left_index=True,right_index=True)
    df_encoder_test = X2_test.copy()#.merge(X2_test,left_index=True,right_index=True)
    df_outs_train = df_encoder_train[['y','y2']]
    df_outs_test = df_encoder_test[['y','y2']]
    df_features_train = df_encoder_train.drop(['error','error2','y','y2'], axis = 1)
    df_features_test = df_encoder_test.drop(['error','error2','y','y2'], axis = 1)
    
#feata = 'BSI_17'

In [ ]:
for feata in new_feat_print:
    #print(feata)
    value = val
    df_features_train = df_features_train[df_features_train[feata] != value]
    df_encoder_train = df_encoder_train[df_encoder_train[feata]!=value]
    
    counter = Model(inputs = [input_dim,input_dim2], outputs = decoded13)
    encoded_train = pd.DataFrame(counter.predict([df_encoder_train,df_features_train]))
    encoded_train = encoded_train.add_prefix('feature_')
    f0_a = encoded_train['feature_0']
    f1_a = encoded_train['feature_1']
    #print(encoded_train['feature_0'].median())
    #print(encoded_train['feature_1'].median())
    df_features_train[feata] = value
    counter = Model(inputs = [input_dim,input_dim2], outputs = decoded13)
    encoded_train = pd.DataFrame(counter.predict([df_encoder_train,df_features_train]))
    encoded_train = encoded_train.add_prefix('feature_')
    f0_d = encoded_train['feature_0']
    f1_d = encoded_train['feature_1']
    #f0r = f0_d - f0_a 
    f0r = f1_d - f1_a 
    pup = len(f0r[f0r>0.001])/len(f0r)
    pdo = len(f0r[f0r<-0.001])/len(f0r)
    peq = len(f0r[abs(f0r)<0.001])/len(f0r)
    
    #print (pup)
    print (pdo)
    #print (peq)
    df_encoder_train = X2_train.copy()#.merge(X2_train,left_index=True,right_index=True)
    df_encoder_test = X2_test.copy()#.merge(X2_test,left_index=True,right_index=True)
    df_outs_train = df_encoder_train[['y','y2']]
    df_outs_test = df_encoder_test[['y','y2']]
    df_features_train = df_encoder_train.drop(['error','error2','y','y2'], axis = 1)
    df_features_test = df_encoder_test.drop(['error','error2','y','y2'], axis = 1)
    
#feata = 'BSI_17'

In [ ]:
for feata in new_feat_print:
    #print(feata)
    value = val
    df_features_train = df_features_train[df_features_train[feata] != value]
    df_encoder_train = df_encoder_train[df_encoder_train[feata]!=value]
    
    counter = Model(inputs = [input_dim,input_dim2], outputs = decoded13)
    encoded_train = pd.DataFrame(counter.predict([df_encoder_train,df_features_train]))
    encoded_train = encoded_train.add_prefix('feature_')
    f0_a = encoded_train['feature_0']
    f1_a = encoded_train['feature_1']
    #print(encoded_train['feature_0'].median())
    #print(encoded_train['feature_1'].median())
    df_features_train[feata] = value
    counter = Model(inputs = [input_dim,input_dim2], outputs = decoded13)
    encoded_train = pd.DataFrame(counter.predict([df_encoder_train,df_features_train]))
    encoded_train = encoded_train.add_prefix('feature_')
    f0_d = encoded_train['feature_0']
    f1_d = encoded_train['feature_1']
    #f0r = f0_d - f0_a 
    f0r = f1_d - f1_a 
    pup = len(f0r[f0r>0.001])/len(f0r)
    pdo = len(f0r[f0r<-0.001])/len(f0r)
    peq = len(f0r[abs(f0r)<0.001])/len(f0r)
    
    #print (pup)
    #print (pdo)
    print (peq)
    df_encoder_train = X2_train.copy()#.merge(X2_train,left_index=True,right_index=True)
    df_encoder_test = X2_test.copy()#.merge(X2_test,left_index=True,right_index=True)
    df_outs_train = df_encoder_train[['y','y2']]
    df_outs_test = df_encoder_test[['y','y2']]
    df_features_train = df_encoder_train.drop(['error','error2','y','y2'], axis = 1)
    df_features_test = df_encoder_test.drop(['error','error2','y','y2'], axis = 1)
    
#feata = 'BSI_17'

In [ ]:
f0_d - f0_a

In [ ]:
df_encoder_train = X2_train.copy()#.merge(X2_train,left_index=True,right_index=True)
df_encoder_test = X2_test.copy()#.merge(X2_test,left_index=True,right_index=True)
df_outs_train = df_encoder_train[['y','y2']]
df_outs_test = df_encoder_test[['y','y2']]
df_features_train = df_encoder_train.drop(['error','error2','y','y2'], axis = 1)
df_features_test = df_encoder_test.drop(['error','error2','y','y2'], axis = 1)

In [ ]:
for feata in new_feat:
    #print(feata)
    value = 5
    df_features_train = df_features_train[df_features_train[feata] != value]
    df_encoder_train = df_encoder_train[df_encoder_train[feata]!=value]
    
    counter = Model(inputs = [input_dim,input_dim2], outputs = decoded13)
    encoded_train = pd.DataFrame(counter.predict([df_encoder_train,df_features_train]))
    encoded_train = encoded_train.add_prefix('feature_')
    f0_a = encoded_train['feature_0'].median()
    f1_a = encoded_train['feature_1'].median()
    #print(encoded_train['feature_0'].median())
    #print(encoded_train['feature_1'].median())
    df_features_train[feata] = value
    counter = Model(inputs = [input_dim,input_dim2], outputs = decoded13)
    encoded_train = pd.DataFrame(counter.predict([df_encoder_train,df_features_train]))
    encoded_train = encoded_train.add_prefix('feature_')
    f0_d = encoded_train['feature_0'].median()
    f1_d = encoded_train['feature_1'].median()
    if f1_a> f1_d:
        print(f1_a,f1_d,feata)
    #print(encoded_train['feature_0'].median())
    #print(encoded_train['feature_1'].median())
    df_encoder_train = X2_train.copy()#.merge(X2_train,left_index=True,right_index=True)
    df_encoder_test = X2_test.copy()#.merge(X2_test,left_index=True,right_index=True)
    df_outs_train = df_encoder_train[['y','y2']]
    df_outs_test = df_encoder_test[['y','y2']]
    df_features_train = df_encoder_train.drop(['error','error2','y','y2'], axis = 1)
    df_features_test = df_encoder_test.drop(['error','error2','y','y2'], axis = 1)
#feata = 'BSI_17'
#feata = 'BSI_46'


In [ ]:
'WHOQoL_1','WHOQoL_6','WHOQoL_9','WHOQoL_14','WHOQoL_19', 'WHOQoL_23',
             'WHOQoL_17', 'WHOQoL_10',
             'BSI_9', 'BSI_49', 'BSI_45','BSI_38','BSI_12', 'BSI_37', 'BSI_23']

In [ ]:
stop

In [ ]:
counter = Model(inputs = [input_dim,input_dim2], outputs = decoded13)
encoded_train = pd.DataFrame(counter.predict([df_encoder_train,df_features_train]))
encoded_train = encoded_train.add_prefix('feature_')
encoded_train['feature_0'].median(),encoded_train['feature_1'].median()

In [ ]:
df_features_train[feata] = 1
counter = Model(inputs = [input_dim,input_dim2], outputs = decoded13)
encoded_train = pd.DataFrame(counter.predict([df_encoder_train,df_features_train]))
encoded_train = encoded_train.add_prefix('feature_')
encoded_train['feature_0'].median(),encoded_train['feature_1'].median()

In [ ]:
df_features_train[feata] = 5
counter = Model(inputs = [input_dim,input_dim2], outputs = decoded13)
encoded_train = pd.DataFrame(counter.predict([df_encoder_train,df_features_train]))
encoded_train = encoded_train.add_prefix('feature_')
encoded_train['feature_0'].median(),encoded_train['feature_1'].median()

In [ ]:
df_encoder_train = X2_train.copy()#.merge(X2_train,left_index=True,right_index=True)
df_encoder_test = X2_test.copy()#.merge(X2_test,left_index=True,right_index=True)
df_outs_train = df_encoder_train[['y','y2']]
df_outs_test = df_encoder_test[['y','y2']]
df_features_train = df_encoder_train.drop(['error','error2','y','y2'], axis = 1)
df_features_test = df_encoder_test.drop(['error','error2','y','y2'], axis = 1)

feata = 'WHOQoL_17'
value = 1
idn = 851
#df_features_train = df_features_train[df_features_train[feata] != value]
#df_encoder_train = df_encoder_train[df_encoder_train[feata]!=value]
    
counter = Model(inputs = [input_dim,input_dim2], outputs = decoded13)
encoded_train = pd.DataFrame(counter.predict([df_encoder_train,df_features_train]))
encoded_train = encoded_train.add_prefix('feature_')
#print(encoded_train['feature_0'].iloc[idn])
#print(encoded_train['feature_1'].iloc[idn])
df_features_train[feata] = value
counter = Model(inputs = [input_dim,input_dim2], outputs = decoded13)
encoded_train = pd.DataFrame(counter.predict([df_encoder_train,df_features_train]))
encoded_train = encoded_train.add_prefix('feature_')
print(encoded_train['feature_0'].iloc[idn])
print(encoded_train['feature_1'].iloc[idn])

In [ ]:
encoded_train = pd.DataFrame(counter.predict([df_encoder_train,df_features_train]))
encoded_train = encoded_train.add_prefix('feature_')
np.mean(encoded_train['feature_0']), np.mean(encoded_train['feature_1'])

In [ ]:
idn = 7825
idn2 = 612
idn3 = 7784


In [ ]:
print(encoded_train['feature_0'].iloc[idn],encoded_train['feature_0'].iloc[idn2],encoded_train['feature_0'].iloc[idn3])
print(encoded_train['feature_1'].iloc[idn],encoded_train['feature_1'].iloc[idn2],encoded_train['feature_1'].iloc[idn3])

In [ ]:


#X2_train.iloc[ind]

In [ ]:
df_encoder_train = X2_train.copy()#.merge(X2_train,left_index=True,right_index=True)
df_encoder_test = X2_test.copy()#.merge(X2_test,left_index=True,right_index=True)
df_outs_train = df_encoder_train[['y','y2']]
df_outs_test = df_encoder_test[['y','y2']]
df_features_train = df_encoder_train.drop(['error','error2','y','y2'], axis = 1)
df_features_test = df_encoder_test.drop(['error','error2','y','y2'], axis = 1)

In [ ]:
feata = 'BSI_10'

df_features_test[feata] = 1

counter = Model(inputs = [input_dim,input_dim2], outputs = encoded13)
#counter = Model(inputs = [input_dim,input_dim2], outputs = decoded13)
encoded_input = Input(shape = (encoding_dim, ))
encoded_test = pd.DataFrame(counter.predict([df_encoder_test,df_features_test]))
encoded_test = encoded_test.add_prefix('feature_')
encoded_test['feature_1'].iloc[[ind]]

In [ ]:
1143   -0.251098
1432   -0.050221
774     0.472253
863     2.349966
380    -0.142225
Name: feature_0, dtype: float32

In [ ]:
['BSI_23','BSI_36','BSI_20','BSI_17','BSI_38','BSI_46','BSI_31','BSI_10','BSI_53',
             'WHOQoL_16','WHOQoL_17','WHOQoL_18','WHOQoL_19','WHOQoL_20','WHOQoL_21','WHOQoL_22',
            'WHOQoL_23','WHOQoL_24','WHOQoL_25','WHOQoL_26']